In [0]:
import pandas as pd
import numpy as np
import requests, json

api_key = '70D413B2-0123-3524-9634-F730BE441009' 

#years used in study
years = map(str,range(1981,2018))

#states used in study
states = ['IL','IN','IA','KS','KY','MI','MO','NE','OH','SD','WI']

payload = {'key':api_key,
          'source_desc':'CENSUS', 
           'sector_desc':'CROPS',
           'group_desc':'FIELD CROPS',
           'commodity_desc':'CORN', 
           'util_practice_desc':'GRAIN', 
           'unit_desc':'ACRES',
           'statisticcat_desc':'AREA HARVESTED',
           'domain_desc':'TOTAL',
           'prodn_practice_desc':'ALL PRODUCTION PRACTICES',
           'agg_level_desc':'COUNTY', 
           'state_alpha':states,
           'year':years,
           'format':'JSON'}

In [0]:
r = requests.get('http://quickstats.nass.usda.gov/api/api_GET/?key='+api_key,params=payload)
df = pd.read_json(json.dumps(r.json()),orient='split') 

In [0]:
#truncate dow to columns of interest
df_trunc = df[['state_fips_code','county_name','county_code','year','Value']]

In [6]:
#find counties with missing data
sum(df_trunc.Value.str.contains('D'))

75

In [8]:
#find original diminensions of df_trunc
#after removing problematic rows we expect 75 less rows than original
df_trunc.shape

(4043, 5)

In [9]:
#remove rows that contain 'D' which is the usda marking for no data.
#.contains returns true for columns that contain 'D', then ~ negates it so we can later on only extract the true columns

df_trunc = df_trunc[~df_trunc.Value.str.contains('D')]
df_trunc.shape

(3968, 5)

In [0]:
#remove comma values and replace with integers
list_area = list(df_trunc['Value'])
list_area = [i.replace(',','') for i in list_area]
df_trunc['Value'] = list_area

In [12]:
#preview df_trunc
df_trunc.head()

,state_fips_code,county_name,county_code,year,Value
0,17,BUREAU,11,2012,270914
1,17,BUREAU,11,2007,307842
2,17,BUREAU,11,2002,251941
3,17,BUREAU,11,1997,257811
4,17,CARROLL,15,2012,149056


In [0]:
#save as csv
df_trunc.to_csv('harvested_area.csv',header=False, index = False)